<a href="https://colab.research.google.com/github/Sivan309/HFAnalyticsCourse/blob/main/Email_Journey_Query_(GA_to_Python).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#Data for Medicare LSE
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = './sivatest-328113-e20c58bed4da.json'
VIEW_ID = '227537175'

def initialize_analyticsreporting():
  credentials = ServiceAccountCredentials.from_json_keyfile_name(
      KEY_FILE_LOCATION, SCOPES)
  analytics = build('analyticsreporting', 'v4', credentials=credentials)
  return analytics

#Get one report page
def get_report(analytics, pageTokenVar):
  return analytics.reports().batchGet(
      body={
        'reportRequests': [
        {
          'viewId': VIEW_ID,
            #Change Date Rate as needed
          'dateRanges': [{'startDate': '2022-02-09', 'endDate': 'yesterday'}],
          'metrics': [{'expression': 'ga:sessions'}, {'expression': 'ga:bounces'}, {'expression': 'ga:pageviews'}, {'expression': 'ga:uniquepageviews'},{'expression': 'ga:timeonpage'}, {'expression': 'ga:avgsessionDuration'}],
          'dimensions': [{'name': 'ga:date'}, {'name': 'ga:campaign'}, {'name': 'ga:pagepath'}, {'name': 'ga:devicecategory'}, {'name': 'ga:usertype'}],
             "dimensionFilterClauses":[
                {
                   "operator":"AND",
                   "filters":[
                      {
                         "dimensionName":"ga:campaign",
                         "operator":"REGEXP",
                         "expressions":[
                            '00152104|00152102|14731901|10112001'
                        ]
                      }
                   ]
                }
             ],
        }]
      }
  ).execute()
    
def handle_report(analytics,pagetoken,rows):  
    response = get_report(analytics, pagetoken)

    #Header, Dimentions Headers, Metric Headers 
    columnHeader = response.get("reports")[0].get('columnHeader', {})
    dimensionHeaders = columnHeader.get('dimensions', [])
    metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])

    #Pagination
    pagetoken = response.get("reports")[0].get('nextPageToken', None)
    
    #Rows
    rowsNew = response.get("reports")[0].get('data', {}).get('rows', [])
    rows = rows + rowsNew
    print("len(rows): " + str(len(rows)))

    #Recursivly query next page
    if pagetoken != None:
        return handle_report(analytics,pagetoken,rows)
    else:
        #nicer results
        nicerows=[]
        for row in rows:
            dic={}
            dimensions = row.get('dimensions', [])
            dateRangeValues = row.get('metrics', [])

            for header, dimension in zip(dimensionHeaders, dimensions):
                dic[header] = dimension

            for i, values in enumerate(dateRangeValues):
                for metric, value in zip(metricHeaders, values.get('values')):
                    if ',' in value or ',' in value:
                        dic[metric.get('name')] = float(value)
                    if '.' in value or '.' in value:
                        dic[metric.get('name')] = str(value)
                    else:
                        dic[metric.get('name')] = int(value)
            nicerows.append(dic)
        return nicerows

#Start
def main():    
    analytics = initialize_analyticsreporting()
    
    global df1
    df1 = []

    rows = []
    rows = handle_report(analytics,'0',rows)

    df1 = pd.DataFrame(list(rows))

if __name__ == '__main__':
  main()


SSLCertVerificationError: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1123)

In [ ]:
import requests as r 
print(r.certs.where())

C:\Anaconda3\lib\site-packages\certifi\cacert.pem


In [ ]:
#rename columns 
df = df1.rename(columns={'ga:bounces':'Bounces','ga:date' : 'Date', 'ga:campaign' : 'Campaign','ga:pagepath': 'Page', 'ga:usertype': 'User Type', 'ga:devicecategory': 'Device Category', 'ga:sessions': 'Sessions', 'ga:bounceRate': 'Bounce Rate', 'ga:pageviews': 'Page Views', 'ga:uniquepageviews' : 'Unique Page Views', 'ga:timeonpage': 'Time on Page', 'ga:avgsessionDuration': 'Avg. Session Duration'})
#Date formatting
df['Date'] = pd.to_datetime(df['Date'])

#Category 

df['Category'] = np.where(df['Campaign'].str.contains('10112001'), 'Medicare', "Non Medicare")
second_column = df.pop('Category')
df.insert(1, 'Category', second_column)
#Tactic Name
df["Tactic Name"] = ""
fourth_column = df.pop('Tactic Name')
df.insert(3, 'Tactic Name', fourth_column)


#Rules
df.loc[df.Campaign.str.contains('10112001'), "Tactic Name"] = "Medicare LC/LSE Email Campaign - Tactic 10112001"
df.loc[df.Campaign.str.contains('00152102'), "Tactic Name"] = "Lead Nurturing - Monthly Newsletter Campaign - Shop IFP ENG - Tactic 00152102"
df.loc[df.Campaign.str.contains('00152104'), "Tactic Name"] = "Lead Nurturing - Monthly Newsletter Campaign - Shop IFP SPA - Tactic 00152104"
df.loc[df.Campaign.str.contains('00152103'), "Tactic Name"] = "Lead Nurturing - Monthly Newsletter Campaign - VCO - Tactic 00152103"
df.loc[df.Campaign.str.contains('00152105'), "Tactic Name"] = "Lead Nurturing - Monthly Newsletter Campaign - VCO SPA - Tactic 00152105"
df.loc[df.Campaign.str.contains('14731901'), "Tactic Name"] = "Lead Nurturing Still Shopping Re-engagement Campaign - Tactic 14731901"
df["VS Visits"] = "0"
df["VS Appointments"] = "0"
df["Events"] = "0"
#Sort
df.sort_values(by='Date', ascending=True)
dfLSE = df
dfLSE


In [ ]:
dfLSE.to_excel (r'C:\Users\snuthanapati\OneDrive - Healthfirst\Desktop\Medicare_LSE0209.xlsx', index = False, header=True)

In [ ]:
#Data for Lead Nurturing Drip 
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = './sivatest-328113-e20c58bed4da.json'
VIEW_ID = '189412434'

def initialize_analyticsreporting():
  credentials = ServiceAccountCredentials.from_json_keyfile_name(
      KEY_FILE_LOCATION, SCOPES)
  analytics = build('analyticsreporting', 'v4', credentials=credentials)
  return analytics

#Get one report page
def get_report(analytics, pageTokenVar):
  return analytics.reports().batchGet(
      body={
        'reportRequests': [
        {
          'viewId': VIEW_ID,
          'dateRanges': [{'startDate': '2022-02-09', 'endDate': 'yesterday'}],
          'metrics': [{'expression': 'ga:sessions'}, {'expression': 'ga:bounces'}, {'expression': 'ga:pageviews'}, {'expression': 'ga:uniquepageviews'},{'expression': 'ga:timeonpage'}, {'expression': 'ga:avgsessionDuration'}],
          'dimensions': [{'name': 'ga:date'}, {'name': 'ga:campaign'}, {'name': 'ga:pagepath'}, {'name': 'ga:devicecategory'}, {'name': 'ga:usertype'}],
             "dimensionFilterClauses":[
                {
                   "operator":"AND",
                   "filters":[
                      {
                         "dimensionName":"ga:campaign",
                         "operator":"REGEXP",
                         "expressions":[
                            'drip'
                        ]
                      }
                   ]
                }
             ],
        }]
      }
  ).execute()
    
def handle_report(analytics,pagetoken,rows):  
    response = get_report(analytics, pagetoken)

    #Header, Dimentions Headers, Metric Headers 
    columnHeader = response.get("reports")[0].get('columnHeader', {})
    dimensionHeaders = columnHeader.get('dimensions', [])
    metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])

    #Pagination
    pagetoken = response.get("reports")[0].get('nextPageToken', None)
    
    #Rows
    rowsNew = response.get("reports")[0].get('data', {}).get('rows', [])
    rows = rows + rowsNew
    print("len(rows): " + str(len(rows)))

    #Recursivly query next page
    if pagetoken != None:
        return handle_report(analytics,pagetoken,rows)
    else:
        #nicer results
        nicerows=[]
        for row in rows:
            dic={}
            dimensions = row.get('dimensions', [])
            dateRangeValues = row.get('metrics', [])

            for header, dimension in zip(dimensionHeaders, dimensions):
                dic[header] = dimension

            for i, values in enumerate(dateRangeValues):
                for metric, value in zip(metricHeaders, values.get('values')):
                    if ',' in value or ',' in value:
                        dic[metric.get('name')] = float(value)
                    if '.' in value or '.' in value:
                        dic[metric.get('name')] = str(value)
                    else:
                        dic[metric.get('name')] = int(value)
            nicerows.append(dic)
        return nicerows

#Start
def main():    
    analytics = initialize_analyticsreporting()
    
    global dfanalytics
    dfanalytics = []

    rows = []
    rows = handle_report(analytics,'0',rows)

    dfanalytics = pd.DataFrame(list(rows))

if __name__ == '__main__':
  main()

In [ ]:
#Rename
df = dfanalytics.rename(columns={'ga:bounces':'Bounces','ga:date' : 'Date', 'ga:campaign' : 'Campaign','ga:pagepath': 'Page', 'ga:usertype': 'User Type', 'ga:devicecategory': 'Device Category', 'ga:sessions': 'Sessions', 'ga:bounceRate': 'Bounce Rate', 'ga:pageviews': 'Page Views', 'ga:uniquepageviews' : 'Unique Page Views', 'ga:timeonpage': 'Time on Page', 'ga:avgsessionDuration': 'Avg. Session Duration'})
#Date
df['Date'] = pd.to_datetime(df['Date'])

#Category
df['Category'] = np.where(df['Campaign'].str.contains('drip'), 'Non Medicare', "Medicare")
second_column = df.pop('Category')
df.insert(1, 'Category', second_column)
#Tactic Name
df["Tactic Name"] = ""
fourth_column = df.pop('Tactic Name')
df.insert(3, 'Tactic Name', fourth_column)


#Rules
df.loc[df.Campaign.str.contains('drip'), "Tactic Name"] = "Lead Nurturing Drip Campaign - Tactic 02032001"
df["VS Visits"] = "0"
df["VS Appointments"] = "0"
df["Events"] = "0"
#Sort
df.sort_values(by='Sessions', ascending=False)
dfDrip = df
dfDrip

In [ ]:
dffinal = dfLSE.append(dfDrip)
dffinal

In [ ]:
dffinal.to_excel (r'C:\Users\snuthanapati\OneDrive - Healthfirst\Desktop\MNM0202.xlsx', index = False, header=True)

In [ ]:
pip install "redshift_connector[full]"

In [ ]:
import redshift_connector
import pandas as pd 
import numpy as np

In [ ]:
conn = redshift_connector.connect(
    host='cdp-redshift-prd.cxkofvnakzzx.us-east-1.redshift.amazonaws.com',
    port=5439,
    database='cdp',
    user='cdp_read_all_act',
    password='r3adDataPROD'
 )

In [ ]:
query = "Select received_at, context_campaign_name, event_text, context_page_url FROM cdp.prod_learn_shop_enroll.tracks where (context_campaign_name like '%10112001%' or context_campaign_name like '%4211501%' or context_campaign_name like '%00152104%' or context_campaign_name like '%00152103%' or context_campaign_name like '%00152105%' or context_campaign_name like '%00152102%' or context_campaign_name like '%02032001%' or context_campaign_name like '%leadnurturingdrip%' or context_campaign_name like '%14731901%') AND EVENT not like '%viewed%'"
Events2 = pd.read_sql(query, conn)
Events2

In [ ]:
query = "Select received_at, context_campaign_name, event_text, context_page_url FROM cdp.prod_healthfirst_org.tracks where (context_campaign_name like '%10112001%' or context_campaign_name like '%4211501%' or context_campaign_name like '%00152104%' or context_campaign_name like '%00152103%' or context_campaign_name like '%00152105%' or context_campaign_name like '%00152102%' or context_campaign_name like '%02032001%' or context_campaign_name like '%leadnurturingdrip%' or context_campaign_name like '%14731901%') AND EVENT not like '%viewed%'"
Events2 = pd.read_sql(query, conn)
Events2